In [ ]:
# Model I/O : Input, Output
    # 1. prompts, language models, output parsers
# Retrieval : 외부 데이터에 접근해서 모델에게 제공.
# Chain
# Agents : chain이 필요한 도구를 선택할 수 있게 만듦.
# Memory : 챗봇에 메모리를 넣는 것
# Callbacks : 모델이 답변을 제공하기 전 실제로 모델이 무엇을 생각하고 있는지 중간단계를 알 수 있게 함. 

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate # template을 string으로 받냐, message로 받냐의 차이
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

# Few shot templat : 모델이 어떻게 대답해야하는지에 대한 예제를 AI 모델에게 주는 과정
# 세부적으로 모델에게 어떻게 해야하는지를 프롬프트로 전달하는 것보다 예제를 보여주는 것이 더 성공적일 것. -> 모델은 텍스트를 만들기 때문. 
# 고객지원 챗봇을 만든다면? : language model에게 어떻게 대응해야하는지는 대화기록을 데이터베이스에 가져와서 fewshot template으로 형식화 시키면 됨. 

# 1. 예제의 형식 지정하기 : 데이터베이스에서 가져올 수도 있기 때문에 예제의 형식을 지정해야함. 
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template="""
    Human: {question}
    AI: {answer}
"""

# 2. 만든 예제 리스트를 fewshotPromptTemplate에 전달하여 어떻게 형식화할지 알려줌.
example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix= "Huamn: What do you know about {country}?", # 형식화된 모든 예제 마지막에 나오는 내용. 따라서 사용자의 질문이 무엇인지 나오게 됨. 
    input_variables=['country'] 
)

chain = prompt | chat 
chain.invoke({"country":"Korea"})

AI: 
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate # template을 string으로 받냐, message로 받냐의 차이
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

# Few shot template : 모델이 어떻게 대답해야하는지에 대한 예제를 AI 모델에게 주는 과정
# 세부적으로 모델에게 어떻게 해야하는지를 프롬프트로 전달하는 것보다 예제를 보여주는 것이 더 성공적일 것. -> 모델은 텍스트를 만들기 때문. 
# 고객지원 챗봇을 만든다면? : language model에게 어떻게 대응해야하는지는 대화기록을 데이터베이스에 가져와서 fewshot template으로 형식화 시키면 됨. 

# 1. 예제의 형식 지정하기 : 데이터베이스에서 가져올 수도 있기 때문에 예제의 형식을 지정해야함. 
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


# 2. 이 나라에 대해 무엇을 알고 있어? 라고 물으면 ai가 "answer"처럼 답변했다고 속이게 하는 것.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# 3. example 목록에 있는 각 예제의 형식을 지정해주는 과정. 이를 위해 위의 ChatPromptTemplate를 사용해 형식을 지정해줌.
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples, # suffix, input_variables가 필요 없음.
)

# 4. 이걸 ChatPromptTemplate를 사용하여 형식을 지정.
final_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a geography expert. You give short answers"),
    example_prompt,
    ("human", "What do you know about {country}?"), # 이제 human이 새로운 질문을 할 것.
    ]
)

chain = final_prompt | chat 
chain.invoke({"country":"Korea"})

# AI 모델에게 전문가라고 말하면 답변이 길게 될 것. 채팅 인터페이스를 사용해서 더욱 길게 말하는 것도 있음.
# You give short answers 라고 시스템 메시지를 설정할 수도 있음. 


        I know this:
        There are two Koreas - North Korea and South Korea
        Capital of South Korea: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='\n        I know this:\n        There are two Koreas - North Korea and South Korea\n        Capital of South Korea: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

In [10]:
# 예제를 선택하는 방법 : LengthBasedExampleSelector
# 예제를 형식화하여 예제의 양이 얼마나 되는지 알려줌. 설정값에 따라 prompt에 알맞은 예제를 골라줌.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# 유저의 로그인 여부, 유저가 사용하는 언어에 따라 examples들을 허용할 수 있음. 
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

 # FewshotPromptTemplate는 리스트에 있는 모든 예제를 가지고 형식화하기 때문에 examples는 지워줌
prompt = FewShotPromptTemplate( 
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix= "Huamn: What do you know about {country}?",
    input_variables=['country'] 
)

prompt.format(country='Brazil')
# max_length=10, 어떤 예시도 선택되지 못함. -> 'Huamn: What do you know about Brazil?'
# max_length=80, france가 선택됨
# max_length=100, france, italy가 선택됨.

# 이를 통해 LLM으로 예제 수를 제한할 수 있음. 비용 절약도 가능! 

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuamn: What do you know about Brazil?'

In [12]:
# 예제를 선택하는 방법 : LengthBasedExampleSelector
# 예제를 형식화하여 예제의 양이 얼마나 되는지 알려줌. 설정값에 따라 prompt에 알맞은 예제를 골라줌.

from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 예시로 example selector 만들기
class RandomExampleSelctor(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)] # example은 list 형식


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# 유저의 로그인 여부, 유저가 사용하는 언어에 따라 examples들을 허용할 수 있음. 
example_selector = RandomExampleSelctor(
    examples=examples,
)

 # FewshotPromptTemplate는 리스트에 있는 모든 예제를 가지고 형식화하기 때문에 examples는 지워줌
prompt = FewShotPromptTemplate( 
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix= "Huamn: What do you know about {country}?",
    input_variables=['country'] 
)

prompt.format(country='Brazil')
# max_length=10, 어떤 예시도 선택되지 못함. -> 'Huamn: What do you know about Brazil?'
# max_length=80, france가 선택됨
# max_length=100, france, italy가 선택됨.

# 이를 통해 LLM으로 예제 수를 제한할 수 있음. 비용 절약도 가능! 

'Human: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuamn: What do you know about Brazil?'

In [15]:
# Serialization and Composition
# 직렬화, 불러오기 저장을 의미 / 작은 prompt를 가지고 이들을 모두 결합하는 것.
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# .json, .yaml 파일로 prompt를 불러오고 포맷팅
# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")


'What is the capital of Germany'

In [19]:
# PipelinePromptTemplate: 많은 prompt들의 memory등을 다 모으는 방법
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final,
                                     pipeline_prompts=prompts,)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrg! That is a secret!! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat
chain.invoke({"character":"Pirate",
              "example_question":"What is your location?",
              "example_answer":"Arrrg! That is a secret!! Arg arg!!",
              "question":"What is your fav food?",})

Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' beats the taste o' the sea! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' beats the taste o' the sea! Arrr!")

In [23]:
# Caching : llm 응답을 저장. 치팅 봇이 항상 똑같은 질문을 받는다면 계속 답변을 만들지 않고 이미 답변한 답을 캐싱을 이용하여 저장하여 재사용 가능!

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache()) # 모든 response가 메모리 저장될 것.
# set_debug(True) # chain 작업할 때 log로 보여주어 편리함.
set_llm_cache(SQLiteCache("cache.db")) # 즉각 cache.db라는 데이터베이스가 생성됨.

chat = ChatOpenAI(
    temperature=0.1,
)

# in memory cache는 컴퓨터를 다시 시작하게 된다면 아까 질문했던 것을 다시 만들어야함.
# 따라서 이럴 때는 데이터베이스에 llm 응답을 저장할 수 있음. 

chat.predict("How do you make Italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [4.64s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it becomes smooth and elastic. If the dough is too dry, you can add a little water to help it come together.\n5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portio

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.\n4. Use your hands to knead the dough until it becomes smooth and elastic. If the dough is too dry, you can add a little water to help it come together.\n5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a rolling pin or pasta machine.\n7. Cut the rolled-out dough into your desired pasta shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for a few minutes un

In [21]:
chat.predict("How do you make Italian pasta")
# 바로 뚱 나오게 됨! LLM에게 물어보지 않고 캐싱을 이용해 답변을 재사용하여 비용을 절약한 것. 

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnj

In [3]:
chat.predict("What do you know about France?")


France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is known for its wine production, with regions such as Bordeaux, Burgundy, and Champagne producing some of the world's most renowned wines. French cuisine is also highly regarded, with dishes such as croissants, baguettes, escargot, and coq au vin being popular both in France and internationally.

The country has a diverse landscape, ranging from the snow-capped Alps in the east to the sandy beaches of the French Riviera in the south. France is also home to numerous UNESCO World Heritage sites, including Mont-Saint-Michel, Versailles Palace, and the Loire Valley.

French is the official language of France, and the country has a population of around 6

"France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is known for its wine production, with regions such as Bordeaux, Burgundy, and Champagne producing some of the world's most renowned wines. French cuisine is also highly regarded, with dishes such as croissants, baguettes, escargot, and coq au vin being popular both in France and internationally.\n\nThe country has a diverse landscape, ranging from the snow-capped Alps in the east to the sandy beaches of the French Riviera in the south. France is also home to numerous UNESCO World Heritage sites, including Mont-Saint-Michel, Versailles Palace, and the Loire Valley.\n\nFrench is the official language of France, and the country has a population of a